Обзор функций

Ниже приведены ключевые столбцы, включенные в очищенный набор данных (неочищенная версия содержит более или менее одинаковые поля):

Product_title - Полное название/название продукта

Product_rating - Средний рейтинг клиентов (числовой) из 5

Total_reviews – Общее количество отзывов клиентов

Purchased_last_month - Единицы, приобретенные в прошлом месяце

Discounted_price - Текущая цена после скидки

Original_price - Первоначальная указанная цена до скидки

Discount_percentage - Процентная скидка, примененная к продукту

Is_best_seller - Указывает, помечен ли продукт как бестселлер

Is_sponsored - Является ли продукт спонсируемым или органическим

Has_coupon - Специальные купоны на скидку (True/False)

Buy_box_availability - Доступность кнопки BuyBox на странице поиска Amazon, как добавить в корзину (значения nan представляют False)

Дата доставки - Предполагаемая дата доставки (преобразуется в формат datetime)

Sustainability_tags - Экологически чистые и связанные с устойчивым развитием теги

Product_image_url – Прямая ссылка на изображение продукта

Product_page_url - Официальный URL-адрес страницы продукта Amazon

Data_collected_at – Дата сбора данных

Product_category - Назначенная категория продукта на основе названия

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier


In [24]:
df = pd.read_csv('/Users/dmitrybertnikov/Desktop/ds_learn/kaggle/amazon_products_sales_data_cleaned.csv')
df.head().T

,0,1,2,3,4
product_title,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...","DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...","Apple AirPods Pro 2 Wireless Earbuds, Active N...",Apple AirTag 4 Pack. Keep Track of and find Yo...
product_rating,4.6,4.3,4.6,4.6,4.8
total_reviews,375.0,2457.0,3044.0,35882.0,28988.0
purchased_last_month,300.0,6000.0,2000.0,10000.0,10000.0
discounted_price,89.68,9.99,314.0,162.24,72.74
original_price,159.0,15.99,349.0,162.24,72.74
is_best_seller,No Badge,No Badge,No Badge,Best Seller,No Badge
is_sponsored,Sponsored,Sponsored,Sponsored,Organic,Organic
has_coupon,Save 15% with coupon,No Coupon,No Coupon,No Coupon,No Coupon
buy_box_availability,Add to cart,Add to cart,Add to cart,NaN,NaN


In [25]:
print(df.shape)
df.info()

(42675, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_title         42675 non-null  object 
 1   product_rating        41651 non-null  float64
 2   total_reviews         41651 non-null  float64
 3   purchased_last_month  32164 non-null  float64
 4   discounted_price      40613 non-null  float64
 5   original_price        40613 non-null  float64
 6   is_best_seller        42675 non-null  object 
 7   is_sponsored          42675 non-null  object 
 8   has_coupon            42675 non-null  object 
 9   buy_box_availability  28022 non-null  object 
 10  delivery_date         30692 non-null  object 
 11  sustainability_tags   3408 non-null   object 
 12  product_image_url     42675 non-null  object 
 13  product_page_url      40606 non-null  object 
 14  data_collected_at     42675 non-null  object 
 15  product

In [26]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
product_rating,41651.0,4.399431,0.386997,1.00,4.20,4.50,4.70,5.00
total_reviews,41651.0,3087.106000,13030.460133,1.00,82.00,343.00,1886.00,865598.00
purchased_last_month,32164.0,1293.665278,6318.323574,50.00,100.00,200.00,400.00,100000.00
discounted_price,40613.0,243.227289,473.351545,2.16,29.69,84.99,224.00,5449.00
original_price,40613.0,257.611107,496.633495,2.16,32.99,89.00,229.99,5449.00
discount_percentage,40613.0,6.547151,12.744715,0.00,0.00,0.00,8.49,85.42


In [28]:
missing_analysis = pd.DataFrame({
    'missing_count': df.isnull().sum(),
    'missing_percentage': (df.isnull().sum() / len(df)) * 100
}).sort_values('missing_percentage', ascending=False)

print("Анализ пропусков:")
print(missing_analysis)

Анализ пропусков:
                      missing_count  missing_percentage
sustainability_tags           39267           92.014060
buy_box_availability          14653           34.336262
delivery_date                 11983           28.079672
purchased_last_month          10511           24.630346
product_page_url               2069            4.848272
discount_percentage            2062            4.831869
discounted_price               2062            4.831869
original_price                 2062            4.831869
total_reviews                  1024            2.399531
product_rating                 1024            2.399531
product_category                  0            0.000000
data_collected_at                 0            0.000000
product_title                     0            0.000000
product_image_url                 0            0.000000
is_sponsored                      0            0.000000
is_best_seller                    0            0.000000
has_coupon                    

In [31]:
df['is_best_seller'].value_counts()

is_best_seller
No Badge             40814
Amazon's               744
Limited time deal      409
Best Seller            275
Save 30%               125
Ends in                 97
Save 10%                61
Save 18%                57
Save 17%                51
Save 12%                26
Save 9%                 14
Save 77%                 2
Name: count, dtype: int64

In [29]:
df_clean = df.copy()

# заполняем медианой по категории пропущенные значения рейтинга
df_clean['product_rating'] = df_clean.groupby('product_category')['product_rating'].transform(
    lambda x: x.fillna(x.median())
)
df_clean['total_reviews'] = df_clean.groupby('product_category')['total_reviews'].transform(
    lambda x: x.fillna(x.median())
)

# Ценновые признаки
df_clean['discounted_price'] = df_clean.groupby('product_category')['discounted_price'].transform(
    lambda x: x.fillna(x.median())
)
df_clean['original_price'] = df_clean['original_price'].fillna(df_clean['discounted_price'])

# Пересчитаем скидку
df_clean['discounted_percentage'] = ((df_clean['original_price'] - df_clean['discounted_price']) / 
                                     df_clean['original_price'] * 100).fillna(0)

high_missing_cols = ['purchased_last_month', 'buy_box_availability', 'delivery_date', 'sustainability_tags']

for col in high_missing_cols:
    df_clean[f'{col}_missing'] = df_clean[col].isnull().astype(int)
    
# Заполняем числовые медианой, категориальные модой
if 'purchased_last_month' in df_clean.columns:
    df_clean['purchased_last_month'] = df_clean['purchased_last_month'].fillna(0)
if 'buy_box_availability' in df_clean.columns:
    df_clean['buy_box_availability'] = df_clean['buy_box_availability'].fillna('Unknown')

In [30]:
df_clean.isna().sum()

product_title                       0
product_rating                      0
total_reviews                       0
purchased_last_month                0
discounted_price                    0
original_price                      0
is_best_seller                      0
is_sponsored                        0
has_coupon                          0
buy_box_availability                0
delivery_date                   11983
sustainability_tags             39267
product_image_url                   0
product_page_url                 2069
data_collected_at                   0
product_category                    0
discount_percentage              2062
discounted_percentage               0
purchased_last_month_missing        0
buy_box_availability_missing        0
delivery_date_missing               0
sustainability_tags_missing         0
dtype: int64

In [ ]:
def map_badge(badge): 
    badge_str = str(badge)
    if badge in ['Best Seller', "Amazon's"]:
        return 'premium'
    if badge == 'Limited time deal':
        return 'promo'
    elif 'Save' in badge_str:
        return 'discount'
    elif badge == 'Ends in':
        return 'urgency'
    else:
        return 'no_badge'
    
df_clean['badge_type'] = df_clean['is_best_seller'].apply(map_badge)
df_clean['is_succes'] = df_clean['badge_type'].isin(['premium', 'promo']).astype(int)

df_clean['price_ratio'] = df_clean['discounted_price'] / df_clean['original_price']
df_clean['reviews_per_price'] = df_clean['total_reviews'] / (df_clean['discounted_price'] + 1)
df_clean['is_discounted'] = (df_clean['discount_percentage'] > 0).astype(int)
df_clean['absolute_discount'] = df_clean['original_price'] - df_clean['discounted_price']
